In [ ]:
import numpy as np
import pandas as pd
import nd2reader
import matplotlib.pyplot as plt
import holoviews as hv
from holoviews.operation.datashader import regrid
import skimage.filters
import skimage.feature
import scipy.ndimage
import peakutils
from tqdm import tnrange, tqdm_notebook
import dask
import dask.array as da
import distributed
from distributed import Client, LocalCluster, progress
from dask_jobqueue import SLURMCluster
from cytoolz import partial, compose
from itertools import repeat
from glob import glob
import cachetools
import numpy_indexed
import pickle
import pyarrow as pa
import warnings
import os
from numbers import Integral
from dask.delayed import Delayed

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from segmentation import *
# from util import *
# from matriarch_stub import *
import segmentation
import matriarch_stub

In [ ]:
plt.rcParams["figure.figsize"] = (8, 8)
hv.extension("bokeh")

# Functions

In [ ]:
def func1(x, a, b, c):
    return a * np.exp(-b * x) + c


def exp_fit(trace):
    y = trace / trace[0]
    ts = np.arange(y.shape[0])
    mask = np.isfinite(y)
    y_finite = y[mask]
    y_min = y_finite.min()
    p, _ = scipy.optimize.curve_fit(
        func1, ts[mask], y_finite, p0=[y_finite[0] - y_min, 0.001, y_min]
    )
    fy = func1(ts, *p)
    return fy

# Load data

In [ ]:
# filename = '/n/groups/paulsson/jqs1/molecule-counting/190522photobleaching_noflatcorr_test1.pickle'
filename = (
    "/n/groups/paulsson/jqs1/molecule-counting/190521photobleaching_noflatcorr.pickle"
)

In [ ]:
data = pickle.load(open(filename, "rb"))

# Debug

In [ ]:
nd2_filename = "/n/scratch2/jqs1/190508/GFP_100ms_100pct.nd2"
channel = "FITC"
# nd2_filename = '/n/scratch2/jqs1/190507/mkate_100ms_100pct.nd2'
# channel = 'mCherry'
idx = 91
pos = 1
d = np.asarray(data[nd2_filename][pos]["traces"]["mean"])
regionprops = data[nd2_filename][pos]["regionprops"]
seg_frame = np.asarray(data[nd2_filename][pos]["segmentation_frame"])
label_frame = np.asarray(data[nd2_filename][pos]["labels"])
y, x = regionprops.loc[idx, :][["centroid_x", "centroid_y"]]
s = 10
x_lim = y_lim = 2048
x, y = int(x), int(y)
x_min = max(x - s, 0)
x_max = min(x + s, x_lim)
y_min = max(y - s, 0)
y_max = min(y + s, y_lim)
t_max = d.shape[1]
seg = seg_frame[y_min:y_max, x_min:x_max]
labels = label_frame[y_min:y_max, x_min:x_max]

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(seg_frame)

## Browse traces

In [ ]:
%%output size=200
hv.HoloMap({idx: hv.Curve(d[idx] / d[idx, 0]) for idx in range(1, d.shape[0] // 5)})

## Browse residuals

In [ ]:
%%output size=200
hv.HoloMap(
    {
        idx: hv.Curve(d[idx] / d[idx, 0] - exp_fit(d[idx]))
        for idx in range(1, d.shape[0] // 5)
    }
)

## Video

In [ ]:
%%output max_frames=2000
hv.HoloMap(
    {
        t: hv.Image(
            segmentation.get_nd2_frame(nd2_filename, pos, channel, t, memmap=True)[
                y_min:y_max, x_min:x_max
            ]
        )
        for t in range(0, 200, 1)
    }
)

In [ ]:
y = d[idx] / d[idx, 0]
fy = exp_fit(y)
plt.plot(y)
plt.plot(fy)

# Spatial

In [ ]:
%%output size=200
cmap = "bmy"
residual_std = [
    np.nanstd(d[idx] / d[idx, 0] - exp_fit(d[idx])) for idx in range(1, d.shape[0])
]
# colors = 1/d[1:,0]
colors = residual_std  # regionprops['centroid_x']
# swap and flip coordinates to follow RevImage convention
regionprops["x"] = regionprops["centroid_y"]
regionprops["y"] = regionprops["centroid_x"]
regionprops["color"] = colors
xy_plot = hv.Points(regionprops, kdims=["x", "y"], vdims=["color"]).options(
    color=hv.dim("color"), size=7, cmap=cmap
)
xy_plot

# Old

In [ ]:
centroid_trace = np.array(
    [
        segmentation.get_nd2_frame(nd2_filename, 0, channel, t)[y, x]
        for t in range(0, t_max, 1)
    ]
)
centroid_trace2 = np.array(
    [
        segmentation.get_nd2_frame(nd2_filename, 0, channel, t)[y + 1, x + 1]
        for t in range(0, t_max, 1)
    ]
)
bg_trace = np.array(
    [
        segmentation.get_nd2_frame(nd2_filename, 0, channel, t)[y + 10, x + 10]
        for t in range(0, t_max, 1)
    ]
)

In [ ]:
plt.plot(centroid_trace)
plt.plot(centroid_trace2)
plt.plot(bg_trace)
plt.plot(np.asarray(data[nd2_filename][0]["traces"]["mean"]).T[:, 1])
plt.plot(np.asarray(data[nd2_filename][0]["traces"]["mean"]).T[:, 500])